In [2]:
print("hello world")

hello world


In [2]:
!pip install torch

Задание 1
Обновите конфигурационный файл так, чтобы в каждом из эмбеддеров обучался последний трансформерный слой/конволюционный блок. 
Запустите обучение на ВМ. При необходимости уменьшите BATCH_SIZE до 4-8. 
После выполнения задания сверьтесь с авторским решением. 

In [ ]:
# Сначала установите и импортируйте torch
!pip install torch torchvision --quiet

import torch
import torchvision
from utils import train

class Config:
    SEED = 42

    TEXT_MODEL_NAME = "bert-base-uncased"
    IMAGE_MODEL_NAME = "tf_efficientnet_b0"

    TEXT_MODEL_UNFREEZE = "encoder.layer.11|pooler"
    IMAGE_MODEL_UNFREEZE = "blocks.6|conv_head|bn2"

    BATCH_SIZE = 256
    TEXT_LR = 3e-5
    IMAGE_LR = 1e-4
    CLASSIFIER_LR = 1e-3
    EPOCHS = 30
    DROPOUT = 0.3
    HIDDEN_DIM = 256
    NUM_CLASSES = 4

    TRAIN_DF_PATH = "data/imdb_train.csv"
    VAL_DF_PATH = "data/imdb_val.csv"
    SAVE_PATH = "best_model.pth"



device = "cuda" if torch.cuda.is_available() else "cpu"
cfg = Config()

train(cfg, device)


Разморожен слой: encoder.layer.11.attention.self.query.weight
Разморожен слой: encoder.layer.11.attention.self.query.bias
Разморожен слой: encoder.layer.11.attention.self.key.weight
Разморожен слой: encoder.layer.11.attention.self.key.bias
Разморожен слой: encoder.layer.11.attention.self.value.weight
Разморожен слой: encoder.layer.11.attention.self.value.bias
Разморожен слой: encoder.layer.11.attention.output.dense.weight
Разморожен слой: encoder.layer.11.attention.output.dense.bias
Разморожен слой: encoder.layer.11.attention.output.LayerNorm.weight
Разморожен слой: encoder.layer.11.attention.output.LayerNorm.bias
Разморожен слой: encoder.layer.11.intermediate.dense.weight
Разморожен слой: encoder.layer.11.intermediate.dense.bias
Разморожен слой: encoder.layer.11.output.dense.weight
Разморожен слой: encoder.layer.11.output.dense.bias
Разморожен слой: encoder.layer.11.output.LayerNorm.weight
Разморожен слой: encoder.layer.11.output.LayerNorm.bias
Разморожен слой: pooler.dense.weight
Раз

Задание 2
И снова идём на ВМ. Используя функцию validate из utils.py, последовательно рассчитайте f1 метрики для модели с размороженными слоями на валидационном датасете в двух сценариях: замена текста на строку "text" и замена картинки на случайный тензор. Для этого адаптируйте код загрузчика данных. 
Сравните полученные значения и сделайте вывод о важности модальностей в этой задаче. Учтите, что не для каждого фильма в датасете есть постер. 
После выполнения задания сверьтесь с авторским решением. 

In [ ]:
from functools import partial

import torch
from torch.utils.data import Dataset, DataLoader

from PIL import Image
import timm
import numpy as np
import pandas as pd

import torchmetrics
from transformers import AutoTokenizer

import albumentations as A

from dataset import get_transforms, collate_fn
from utils import MultimodalModel, validate, seed_everything


class Config:
    SEED = 42

    TEXT_MODEL_NAME = "bert-base-uncased"
    IMAGE_MODEL_NAME = "tf_efficientnet_b0"

    TEXT_MODEL_UNFREEZE = "encoder.layer.11|pooler"
    IMAGE_MODEL_UNFREEZE = "blocks.6|conv_head|bn2"

    BATCH_SIZE = 256
    TEXT_LR = 3e-5
    IMAGE_LR = 1e-4
    CLASSIFIER_LR = 1e-3
    EPOCHS = 30
    DROPOUT = 0.3
    HIDDEN_DIM = 256
    NUM_CLASSES = 4

    TRAIN_DF_PATH = "data/imdb_train.csv"
    VAL_DF_PATH = "data/imdb_val.csv"
    SAVE_PATH = "best_model.pth"


class MultimodalDataset(Dataset):

    def __init__(self, config, transforms, ds_type="train", mask="image"):
        if ds_type == "train":
            self.df = pd.read_csv(config.TRAIN_DF_PATH)
        else:
            self.df = pd.read_csv(config.VAL_DF_PATH)
        self.image_cfg = timm.get_pretrained_cfg(config.IMAGE_MODEL_NAME)
        self.tokenizer = AutoTokenizer.from_pretrained(config.TEXT_MODEL_NAME)
        self.transforms = transforms
        self.mask = mask

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        text = self.df.loc[idx, "text"]
        label = self.df.loc[idx, "label"]

        img_path = self.df.loc[idx, "movie_id"]
        try:
            image = Image.open(f"data/imdb_images/{img_path}.jpg").convert('RGB')
            if self.mask == "image":
                image = torch.randint(0, 255, (*self.image_cfg.input_size[1:],
                                           self.image_cfg.input_size[0])).to(
                                               torch.float32)
            else:
                text = "text"
        except:
            image = torch.randint(0, 255, (*self.image_cfg.input_size[1:],
                                           self.image_cfg.input_size[0])).to(
                                               torch.float32)

        image = self.transforms(image=np.array(image))["image"]
        return {"label": label, "image": image, "text": text}


config = Config()
device = "cuda" if torch.cuda.is_available() else "cpu"
seed_everything(config.SEED)

model = MultimodalModel(config).to(device)
state_dict = torch.load("best_model.pth")
model.load_state_dict(state_dict)

tokenizer = AutoTokenizer.from_pretrained(config.TEXT_MODEL_NAME)


transforms = get_transforms(config, ds_type="val")
dataset_no_image = MultimodalDataset(config, transforms, ds_type="val", mask="image")
loader = DataLoader(dataset_no_image,
                            batch_size=config.BATCH_SIZE,
                            shuffle=False,
                            collate_fn=partial(collate_fn,
                                               tokenizer=tokenizer))

f1_metric_val = torchmetrics.F1Score(
        task="binary" if config.NUM_CLASSES == 2 else "multiclass",
        num_classes=config.NUM_CLASSES).to(device)

f1_no_image = validate(model, loader, device, f1_metric_val)

f1_metric_val.reset()

dataset_no_text = MultimodalDataset(config, transforms, ds_type="val", mask="text")
loader = DataLoader(dataset_no_text,
                            batch_size=config.BATCH_SIZE,
                            shuffle=False,
                            collate_fn=partial(collate_fn,
                                               tokenizer=tokenizer))

f1_no_text = validate(model, loader, device, f1_metric_val)
f1_metric_val.reset()

print(f"F1-score, images masked: {f1_no_image:.3f}")
print(f"F1-score, texts masked: {f1_no_text:.3f}")